In [1]:
import jieba

jieba.set_dictionary('data/dict.txt.big')

text = "我喜歡使用Python進行資料分析"

# 搜索引擎模式（適合建立索引）
words = jieba.cut_for_search(text)
print("/".join(words))

c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from c:\Users\nick\OneDrive\stock-llm-neuralprophet\data\dict.txt.big ...
Loading model from cache C:\Users\nick\AppData\Local\Temp\jieba.u79c949ab01ef58952af3f13a7e93c7df.cache
Loading model cost 0.575 seconds.
Prefix dict has been built successfully.


我/喜歡/使用/Python/進行/資料/分析


In [2]:
import tiktoken

# Choose the encoding based on your model
enc = tiktoken.encoding_for_model("gpt-4o")  # or "gpt-3.5-turbo"

In [3]:
# Encode to token IDs
tokens = enc.encode(text)

print(tokens)       # List of token IDs
print(len(tokens))  # Token count

tokens_str = [enc.decode([t]) for t in tokens]
print(tokens_str)

[7522, 30734, 141775, 26019, 60502, 77897, 6550, 118599, 32648]
9
['我', '喜', '歡', '使用', 'Python', '進', '行', '資料', '分析']


In [4]:
def split_text(text):
    return jieba.cut_for_search(text)
    
    # Encode to token IDs
    # tokens = enc.encode(text)
    # tokens_str = [enc.decode([t]) for t in tokens]
    # return tokens_str

In [5]:
import pandas as pd

news_df = pd.read_csv('data/news/news_2022-11-01.csv', parse_dates=True)[['content', 'publish_at']]

news_df

,content,publish_at
0,關注台股盤前要聞重點，台經院昨 (31) 日公布製造業個別產業景氣信號，9 月製造業景氣信號...,2022-11-01 05:10:03
1,在一連串經濟數據不佳後，市場認為美通膨可能放緩，美 Fed 立場有機會不再如此偏鷹，11 月...,2022-11-01 08:00:06
2,今日為國巨 (2327-TW) 減資新股恢復交易日，國巨收盤差一檔即漲停版，收 367 元，...,2022-11-01 08:00:08
3,台北股市昨日大盤上漲 161.33 點作收，收盤報 12949.75 點，漲幅 1.26%，...,2022-11-01 08:02:21
4,美國擴大對中國晶片制裁，主要限制中國先進製程的發展，以及與 HPC 晶片的限制，晶片禁令究竟...,2022-11-01 09:00:09
...,...,...
74,外資今 (1) 日賣超台股 10.68 億元，由買轉賣，台指期現貨淨多單也減少 2702 口...,2022-11-01 19:50:05
75,雲端 POS(PoS / 銷售點管理) 市場，預計從 2020 年到 2025 年的預測期間...,2022-11-01 20:00:06
76,背光模組廠中光電 (5371-TW) 受惠微投影機需求帶動下，今年影像產品事業群出貨可望大增...,2022-11-01 20:20:04
77,工業電腦廠融程電 (3416-TW) 今 (1) 日公告第三季財報，稅後純益 1.62 億元...,2022-11-01 20:30:11


In [6]:
STOCK_ID = 2330

stock_df = pd.read_csv(f'data/stocks/{STOCK_ID}_stock_data_0630.csv', parse_dates=True, index_col=0)

stock_df['increase'] = (stock_df['y'] > stock_df['y'].shift()).shift(-1)
stock_df.dropna(inplace=True)

stock_df = stock_df[['ds', 'y', 'increase']]

stock_df.head(10)

C:\Users\nick\AppData\Local\Temp\ipykernel_74600\3807486973.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_df = pd.read_csv(f'data/stocks/{STOCK_ID}_stock_data_0630.csv', parse_dates=True, index_col=0)


,ds,y,increase
0,2022-11-01,391.5,True
1,2022-11-02,395.0,False
2,2022-11-03,384.0,False
3,2022-11-04,382.0,True
4,2022-11-07,390.0,True
5,2022-11-08,399.0,True
6,2022-11-09,417.0,False
7,2022-11-10,407.5,True
8,2022-11-11,441.5,True
9,2022-11-14,445.0,True


In [7]:
inc_news_arr = []
dec_news_arr = []

for i, row in stock_df.iterrows():
    try:
        news_df = pd.read_csv(f'data/news/news_{row["ds"]}.csv', parse_dates=True)
    except FileNotFoundError:
        print(f"File not found for date: {row['ds']}")
        continue
    
    if row['increase']:
        inc_news_arr.append(news_df)
    else:
        dec_news_arr.append(news_df)
    
inc_news = pd.concat(inc_news_arr, ignore_index=True)
inc_news.to_csv('inc_news.csv', index=False)

dec_news = pd.concat(dec_news_arr, ignore_index=True)
dec_news.to_csv('dec_news.csv', index=False)

File not found for date: 2023-07-03
File not found for date: 2023-07-04
File not found for date: 2023-07-05
File not found for date: 2023-07-06
File not found for date: 2023-07-07
File not found for date: 2023-07-10
File not found for date: 2023-07-11
File not found for date: 2023-07-12
File not found for date: 2023-07-13
File not found for date: 2023-07-14
File not found for date: 2023-07-17
File not found for date: 2023-07-18
File not found for date: 2023-07-19
File not found for date: 2023-07-20
File not found for date: 2023-07-21


In [8]:
import re

def clean_news(doc):
    # Remove line breaks and extra spaces
    pattern = r'https?://[A-Za-z0-9\-._~:/?#\[\]@!$&\'()*+,;=%]+'
    cleaned_text = re.sub(pattern, '', doc).strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def split_news(news):
    splited_news = []
    for i, row in news.iterrows():
        print(f"Processing news {i+1}/{len(news)}")
        text = row['content']
        text = clean_news(text)
        text = re.sub(r'\d+', ' ', text)
        words = split_text(text)
        clean_words = [w for w in words if re.match(r'[\u4e00-\u9fffA-Za-z0-9]+$', w)]
        splited_news.append(list(clean_words))
    return splited_news

split_news(inc_news[:1])

Processing news 1/1


[['關注',
  '台股',
  '盤前',
  '要聞',
  '重點',
  '台',
  '經院',
  '昨',
  '日',
  '公布',
  '製造',
  '製造業',
  '個別',
  '產業',
  '景氣',
  '信號',
  '月',
  '製造',
  '製造業',
  '景氣',
  '信號',
  '值降',
  '至',
  '分',
  '連',
  '個',
  '月亮',
  '起',
  '代表',
  '景氣',
  '低迷',
  '的',
  '黃',
  '藍燈',
  '燈號',
  '值',
  '更',
  '逼近',
  '衰退',
  '藍燈',
  '台積電',
  '奈米',
  '量產',
  '時程',
  '受',
  '客戶',
  '推出',
  '產品',
  '進度',
  '影響',
  '而',
  '延後',
  '傳',
  '逐步',
  '下調',
  '供應',
  '供應商',
  '訂單',
  '較',
  '年初',
  '預估',
  '數量',
  '減少',
  '成',
  '以下',
  '是',
  '今',
  '日必',
  '看',
  '重要',
  '財經',
  '新聞',
  '財經新聞',
  '台',
  '經院',
  '月',
  '製造',
  '製造業',
  '景氣',
  '燈號',
  '續亮',
  '低迷',
  '黃',
  '藍燈',
  '瀕',
  '衰退',
  '藍燈',
  '邊緣',
  '台',
  '經院',
  '昨',
  '日',
  '公布',
  '製造',
  '製造業',
  '個別',
  '產業',
  '景氣',
  '信號',
  '月',
  '製造',
  '製造業',
  '景氣',
  '信號',
  '值降',
  '至',
  '分',
  '不僅',
  '連續',
  '個',
  '月亮',
  '起',
  '代表',
  '景氣',
  '低迷',
  '的',
  '黃',
  '藍燈',
  '燈號',
  '值',
  '更',
  '逼近',
  '衰退',
  '的',
  '藍燈',
  '閱讀',
  '全文',
  '台積電',
  '奈

In [9]:
inc_splited_news = split_news(inc_news)
dec_splited_news = split_news(dec_news)
len(inc_splited_news), len(dec_splited_news)

Processing news 1/23872
Processing news 2/23872
Processing news 3/23872
Processing news 4/23872
Processing news 5/23872
Processing news 6/23872
Processing news 7/23872
Processing news 8/23872
Processing news 9/23872
Processing news 10/23872
Processing news 11/23872
Processing news 12/23872
Processing news 13/23872
Processing news 14/23872
Processing news 15/23872
Processing news 16/23872
Processing news 17/23872
Processing news 18/23872
Processing news 19/23872
Processing news 20/23872
Processing news 21/23872
Processing news 22/23872
Processing news 23/23872
Processing news 24/23872
Processing news 25/23872
Processing news 26/23872
Processing news 27/23872
Processing news 28/23872
Processing news 29/23872
Processing news 30/23872
Processing news 31/23872
Processing news 32/23872
Processing news 33/23872
Processing news 34/23872
Processing news 35/23872
Processing news 36/23872
Processing news 37/23872
Processing news 38/23872
Processing news 39/23872
Processing news 40/23872
Processin

(23872, 22649)

In [10]:
from collections import Counter

def get_word_percentages(words):
    counts = Counter(words)
    total = sum(counts.values())
    percentages = {word: (count / total * 100) for word, count in counts.items()}
    return percentages

wperc_inc = get_word_percentages([word for row in inc_splited_news for word in row])
wperc_dec = get_word_percentages([word for row in dec_splited_news for word in row])

# for word, pct in wperc_inc.items():
#     print(f"{word}: {pct:.2f}%")
# for word, pct in wperc_dec.items():
#     print(f"{word}: {pct:.2f}%")

In [11]:
words = [word for row in inc_splited_news for word in row]
# words[:10]

GRAM = 2

def sliding_windows_words(arr, window_size, step=1):
    return [''.join(arr[i:i+window_size]) for i in range(0, len(arr)-window_size+1, step)]

sliding_windows_words(words[:20], GRAM)

['關注台股',
 '台股盤前',
 '盤前要聞',
 '要聞重點',
 '重點台',
 '台經院',
 '經院昨',
 '昨日',
 '日公布',
 '公布製造',
 '製造製造業',
 '製造業個別',
 '個別產業',
 '產業景氣',
 '景氣信號',
 '信號月',
 '月製造',
 '製造製造業',
 '製造業景氣']

In [12]:
from collections import Counter

def get_word_percentages(words):
    counts = Counter(words)
    total = sum(counts.values())
    percentages = {word: (count / total * 100) for word, count in counts.items()}
    return percentages

wperc_inc = get_word_percentages(sliding_windows_words([word for row in inc_splited_news for word in row], GRAM))
wperc_dec = get_word_percentages(sliding_windows_words([word for row in dec_splited_news for word in row], GRAM))

for word, pct in list(wperc_inc.items())[:10]:
    print(f"{word}: {pct:.2f}%")
for word, pct in list(wperc_dec.items())[:10]:
    print(f"{word}: {pct:.2f}%")

關注台股: 0.00%
台股盤前: 0.00%
盤前要聞: 0.00%
要聞重點: 0.00%
重點台: 0.00%
台經院: 0.00%
經院昨: 0.00%
昨日: 0.01%
日公布: 0.03%
公布製造: 0.00%
關注台股: 0.00%
台股盤前: 0.00%
盤前要聞: 0.00%
要聞重點: 0.00%
重點聯發科: 0.00%
聯發科旗艦: 0.00%
旗艦晶片: 0.00%
晶片下: 0.00%
下周二: 0.00%
周二亮相: 0.00%


In [13]:
def get_perc_diff(counts_a, counts_b):
    # Combine all words
    all_words = set(counts_a) | set(counts_b)
    # Calculate absolute differences
    diff = {
        word: counts_a.get(word, 0) - counts_b.get(word, 0)
        # word: counts_a.get(word, 0) / counts_b.get(word, 0.000000000001)
        for word in all_words if len(word) > 1
    }
    # Sort by biggest difference
    return sorted(diff.items(), key=lambda x: x[1], reverse=True)
    

sorted_diff_inc = get_perc_diff(wperc_inc, wperc_dec)
sorted_diff_dec = get_perc_diff(wperc_dec, wperc_inc)

for i in sorted_diff_inc[:100]:
    print(f"{i[0]}: {i[1]:.4f}")

月營收: 0.0449
營收速報: 0.0242
營收億元: 0.0218
年增率高達: 0.0172
億元年增率: 0.0159
超張: 0.0156
買賣超: 0.0150
高達營收: 0.0118
年增: 0.0097
近日: 0.0086
營收年: 0.0084
月增率: 0.0082
營收為: 0.0081
最新相關: 0.0079
相關新聞: 0.0078
前月: 0.0072
稅後純益: 0.0072
年增率月: 0.0070
大幅成長: 0.0068
增大幅: 0.0067
億億: 0.0063
年月: 0.0062
億元年增: 0.0060
萬萬: 0.0059
第三三季: 0.0052
三季第三季: 0.0052
純益億元: 0.0050
成長營收: 0.0049
同期新高: 0.0049
億元累計: 0.0048
萬元年增率: 0.0047
營收一覽: 0.0047
三季前三季: 0.0046
日股價: 0.0043
為新: 0.0042
為億元: 0.0041
個月: 0.0041
註數據: 0.0041
為投資: 0.0041
近個: 0.0040
張外資: 0.0039
外資買賣: 0.0039
價為元: 0.0039
張自營: 0.0039
張投信: 0.0039
月累計: 0.0039
累計年增率: 0.0039
業務為: 0.0038
最新價為: 0.0038
一覽月份: 0.0038
合計買賣: 0.0038
日籌碼: 0.0038
籌碼三大: 0.0038
張近: 0.0038
投信買賣: 0.0038
自營商買賣: 0.0038
高達註: 0.0038
TW所屬: 0.0038
僅供參考不作: 0.0037
所屬產業: 0.0037
不作為: 0.0037
投資建議: 0.0037
產業為: 0.0037
數據僅供: 0.0037
億元季增: 0.0037
營收年增率: 0.0037
供參考僅供參考: 0.0036
僅供參考: 0.0036
三大法人: 0.0036
個百分: 0.0035
新聞營收: 0.0035
元近: 0.0035
主要業務: 0.0034
營收萬元: 0.0034
自營營商: 0.0033
營商自營商: 0.0033
金TW: 0.0033
累計營收: 0.0033
參考供參考: 0.0032
增率今年

In [14]:
rules = []

pos_sena = [f'如果新聞出現 {i} 的情況，則股票隔日的收盤價可能 上漲' for i in [
    '營收增加',
    '獲利提升',
    '擴大投資',
    '獲得重大訂單或合作案',
    '技術突破或取得專利',
    '策略聯盟或併購案完成',
    '國際關稅降低',
    '匯率有利公司營收(這個要看每間公司進出口不同，像是如果本國貨幣貶值有利出口)',
    '營收、獲利、毛利/營益率提升；財測上修、財報優於預期',
    '重大訂單/長約到手、能見度提升；大客戶拉貨、出貨創高',
    '擴產落地且稼動率上升；良率提升；量產里程碑達成；庫存去化完成',
    '產品漲價且需求穩定；關鍵原料/代工/運費下滑；匯率淨利有利',
    '技術突破、專利取得；新標準被採用；法規/國際認證通過',
    '策略聯盟/併購案完成且綜效明確；政府大標得標',
    '股票回購；信用評等上調；被動指數/ADR/ETF 納入；分拆/子公司上市釋放估值',
    '產業景氣回升、報價上行；競品出事帶來轉單',
    '政策補貼/稅優落地；關稅降低/豁免；併購審查通過',
    '降息或金融條件轉鬆；通膨回落且需求穩；PMI/零售等領先指標轉好',
    '大股東/內部人增持；接班順利/關鍵高層延任'
]]

neg_sena = [f'如果新聞出現 {i} 的情況，則股票隔日的收盤價可能 下跌' for i in [
    '營收下降',
    '獲利減少或虧損',
    '產品銷售不佳或延遲上市',
    '高階管理人員異動或內部動盪',
    '財務危機或債務違約',
    '國際關稅提高',
    '匯率波動(跟上面一樣，如果升值太快像前陣子台幣，對出口負面影響較大)',
    '經濟衰退或成長放緩(通常是經濟指標數據不好)',
    '營收下降、虧損或利潤下滑；毛利/營益率惡化；財測下修、財報不及預期',
    '砍單、能見度下降；擴產過度/稼動不足；良率瓶頸；量產延後；庫存攀升',
    '價格戰降價；原物料/代工/運費上漲；油電價攀升；匯率不利或波動飆升',
    '技術/專利落後；標準被邊緣化；法規/認證駁回；重大資安事故/服務中斷',
    '高集中客戶轉單；政府標案流標/取消；聯盟破局；併購溢價過高、整合不順',
    '信用評等下調；被動指數剔除',
    '產業下行、同業價格戰；勞資糾紛/罷工；天災/事故停工、限水限電',
    '關稅提高；出口管制/反壟斷/反補貼收緊；海關/檢疫加嚴；併購被否決',
    '升息超預期；通膨黏著且需求轉弱；經濟衰退/數據轉差；金融系統性風險；地緣政治升級',
    '高層異動頻繁/內鬥；內部人減持或高比例質押；經營權之爭；ESG/環保違規與罰款'
]]

rules = pos_sena + neg_sena

for word, diff in sorted_diff_inc[:80]:
    rules.append(f"如果關鍵字 {word} 出現，則股票隔日的收盤價可能 上漲")

# 跌的關鍵字比較沒有規律，所以只用幾個
for word, diff in sorted_diff_dec[:20]:
    rules.append(f"如果關鍵字 {word} 出現，則股票隔日的收盤價可能 下跌")
    
rules

['如果新聞出現 營收增加 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 獲利提升 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 擴大投資 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 獲得重大訂單或合作案 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 技術突破或取得專利 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 策略聯盟或併購案完成 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 國際關稅降低 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 匯率有利公司營收(這個要看每間公司進出口不同，像是如果本國貨幣貶值有利出口) 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 營收、獲利、毛利/營益率提升；財測上修、財報優於預期 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 重大訂單/長約到手、能見度提升；大客戶拉貨、出貨創高 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 擴產落地且稼動率上升；良率提升；量產里程碑達成；庫存去化完成 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 產品漲價且需求穩定；關鍵原料/代工/運費下滑；匯率淨利有利 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 技術突破、專利取得；新標準被採用；法規/國際認證通過 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 策略聯盟/併購案完成且綜效明確；政府大標得標 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 股票回購；信用評等上調；被動指數/ADR/ETF 納入；分拆/子公司上市釋放估值 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 產業景氣回升、報價上行；競品出事帶來轉單 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 政策補貼/稅優落地；關稅降低/豁免；併購審查通過 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 降息或金融條件轉鬆；通膨回落且需求穩；PMI/零售等領先指標轉好 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 大股東/內部人增持；接班順利/關鍵高層延任 的情況，則股票隔日的收盤價可能 上漲',
 '如果新聞出現 營收下降 的情況，則股票隔日的收盤價可能 

In [15]:
import pandas as pd
import uuid
from datetime import date

# Your array of words
words = rules

# Generate one UUID (same for all rows)
# unique_id = str(uuid.uuid4())

# Use fixed publish date
publish_date = date.today().isoformat()  # e.g., '2025-07-28'

# Create DataFrame
df = pd.DataFrame({
    "publish_at": publish_date,
    "content": words
})

df["uuid"] = df.apply(
    lambda row: uuid.uuid5(uuid.NAMESPACE_DNS, str(row.content)),
    axis=1
)

df.to_csv('rules.csv', index=False)

df

,publish_at,content,uuid
0,2025-09-07,如果新聞出現 營收增加 的情況，則股票隔日的收盤價可能 上漲,9fb6d87f-229a-523d-a864-138ae95fa683
1,2025-09-07,如果新聞出現 獲利提升 的情況，則股票隔日的收盤價可能 上漲,4c0f97eb-2bc8-5d4d-96fb-58824bfe0405
2,2025-09-07,如果新聞出現 擴大投資 的情況，則股票隔日的收盤價可能 上漲,a6fba059-30f0-5b51-8844-9dbec2db00e5
3,2025-09-07,如果新聞出現 獲得重大訂單或合作案 的情況，則股票隔日的收盤價可能 上漲,c0ff9faf-047b-51c3-8169-69686370f0ef
4,2025-09-07,如果新聞出現 技術突破或取得專利 的情況，則股票隔日的收盤價可能 上漲,0c37ba0c-97ea-5284-b9fc-e83c564d3c9a
...,...,...,...
132,2025-09-07,如果關鍵字 光寶科 出現，則股票隔日的收盤價可能 下跌,0b8249ee-f897-58d9-8f59-f322de977027
133,2025-09-07,如果關鍵字 股東股東會 出現，則股票隔日的收盤價可能 下跌,87e9eae2-8d5f-5784-b01b-f31cda05c509
134,2025-09-07,如果關鍵字 人機無人機 出現，則股票隔日的收盤價可能 下跌,8aa5a766-d59f-5236-9919-5f47e8bfb612
135,2025-09-07,如果關鍵字 無人人機 出現，則股票隔日的收盤價可能 下跌,97ec2a96-ff56-587b-ae7b-5b0e70ee701c
